In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.auto import tqdm
import shutil as sh

In [14]:
FOLDS = 5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [7]:
df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxes = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxes[:, i]

df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w'] / 2
df['y_center'] = df['y'] + df['h'] / 2
df['classes'] = 0
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

df.head()

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0


In [9]:
index = list(set(df.image_id))

['610d45d1e', 'ffbfe7cc0', '939c5d28b', '134d4a01c', '2d6b777d8', '2c28cb6dd', 'fc6860020', '6b055e6cc', '9988ee066', '6b2a72b4f', '76bd18142', 'bd9083329', 'd2265cb91', 'e840ecd4e', '633807391', 'a4dd288c0', 'b9c9c2559', 'd89f4ea06', 'b0caf416d', 'a6afb2315', '2b32a4cc4', '55deda461', 'fa55f0519', '77d1a2316', '84d4ca3d3', 'ba586e514', '3f2eddbe8', 'dd1b2f39a', 'f251e40f2', '8ffe2f23a', 'eef9bcaf5', '725c243a7', 'bb808af6f', '09ba65605', '24a832fa5', '516218913', 'd8cae4d1b', '9d9b579c6', '6f9bcd471', '96efbda67', 'f7d780303', '26a2693ec', 'a1fcb07dd', 'b75c3bb71', '1f7600e2c', '4132df01d', '75791012a', '5a8f53092', 'aaf9d5f5c', '2de7cc0c9', 'd00a142f1', '0cbe3f32c', '2443092e6', 'e2018867c', 'c02bfe3f2', '0989b2627', '6db264b94', '537395836', '9ef3c0a46', '16e075fbb', '2b7b5621c', '85c3c061a', 'f9b92c9e9', '037024e13', '614640a0b', '131708dfd', '6b0b7bbd9', '0e90d45c6', 'ba35357cb', '8a7324c84', '305bf8b23', '6b62a043a', 'e4c9b3866', '6f10a6f8f', '1da65492e', 'c6ca898c2', 'd6a22baf0'

In [26]:
source = 'train'
if True:
    len_fold = len(index) // FOLDS
    for fold in range(FOLDS):
        val_index = index[len_fold * fold: len_fold * (fold + 1)]
        for name, mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path_to_save = 'val2017/'
            else:
                path_to_save = 'train2017/'

            if not os.path.exists('convertor/fold{}/labels/'.format(fold) + path_to_save):
                    os.makedirs('convertor/fold{}/labels/'.format(fold) + path_to_save)
            with open('convertor/fold{}/labels/'.format(fold) + path_to_save + name + ".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold, path_to_save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold, path_to_save))
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path_to_save,name))

  0%|          | 0/3373 [00:00<?, ?it/s]

  0%|          | 0/3373 [00:00<?, ?it/s]

  0%|          | 0/3373 [00:00<?, ?it/s]

  0%|          | 0/3373 [00:00<?, ?it/s]

  0%|          | 0/3373 [00:00<?, ?it/s]

In [25]:
!python train.py --img 1024 --batch 8 --epochs 15 --data input/config-yolo5/wheat0.yaml --cfg input/config-yolo5/yolov5x.yaml --name yolov5x_fold0